# spam classification

POINT TO REMEMBER
- if your data contains only text then you cannot apply machine learning model directly on it.
- you have to convert it into numerical form.
- You will perform Feature extraction on the Dataset
- Feature Extraction is the process of converting data into numerical form.
 - - In scikit-learn, we have 2 classes for this purpose
- - CountVectorizer
- - TfidfVectorizer


# CountVectorizer Demo

In [1]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [2]:
data = [
    'This is the first document.',
    'This document is the second document.',
    'And this is the third one.',
    'Is this the first document?',
]
vec = CountVectorizer()
X = vec.fit_transform(data)

In [3]:
X.toarray()

array([[0, 1, 1, 1, 0, 0, 1, 0, 1],
       [0, 2, 0, 1, 0, 1, 1, 0, 1],
       [1, 0, 0, 1, 1, 0, 1, 1, 1],
       [0, 1, 1, 1, 0, 0, 1, 0, 1]], dtype=int64)

In [4]:
print(vec.get_feature_names_out())

['and' 'document' 'first' 'is' 'one' 'second' 'the' 'third' 'this']


In [5]:
print(vec.vocabulary_)

{'this': 8, 'is': 3, 'the': 6, 'first': 2, 'document': 1, 'second': 5, 'and': 0, 'third': 7, 'one': 4}


In [6]:
print(vec.transform(["this is a test document"]).toarray())
print(vec.transform(["eat your food"]).toarray())
print(vec.transform(["document this, document that, everything is a document"]).toarray())

[[0 1 0 1 0 0 0 0 1]]
[[0 0 0 0 0 0 0 0 0]]
[[0 3 0 1 0 0 0 0 1]]


# TfidfVectorizer Demo
formula for tfidf is

- term frequency = (Number of times term t appears in a document)/(Number of terms in the document)
- inverse document frequency = log(N/n), where, N is the number of documents and n is the number of documents a term t has appeared in.
- TF-IDF = term frequency * inverse document frequency

In [7]:

import numpy as np

np.set_printoptions(precision=2) # to make output more readable